In [1]:
import pandas as pd
from keras.utils import to_categorical
import random
import numpy as np
import os
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D, MaxPooling2D, Dropout, Activation, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img, array_to_img, img_to_array
from tensorflow.keras.models import Sequential
from glob import glob
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, Input, TimeDistributed
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.applications import MobileNet
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
zip1 = zipfile.ZipFile('/content/drive/MyDrive/DR/MultispectralColourEq.zip')
list1= zip1.namelist()
with zipfile.ZipFile('/content/drive/MyDrive/DR/MultispectralColourEq.zip',"r") as zip_ref:
    z1=zip_ref.extractall()

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [5]:
pip install split-folders

In [6]:
import splitfolders
source_folder = '/content/MultispectralColourEq'


output_folder = 'split_data2'


splitfolders.ratio(source_folder, output=output_folder, ratio=(0.7, 0.3))

print("Data splitting completed.")

Copying files: 1000 files [00:00, 2055.10 files/s]

Data splitting completed.


In [7]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    strategy = tf.distribute.OneDeviceStrategy("/device:GPU:0")  # Use the first available GPU (T4 GPU)
else:
    strategy = tf.distribute.OneDeviceStrategy("/device:CPU:0")

In [8]:
with strategy.scope():
    train_datagen = ImageDataGenerator(rescale=1.0/255.0)
    test_datagen = ImageDataGenerator(rescale=1.0/255.0)

    train_dir = '/content/split_data2/train'
    test_dir = '/content/split_data2/val'

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',  # Multi-class classification
    )

    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',  # Multi-class classification
    )

Found 700 images belonging to 4 classes.
Found 300 images belonging to 4 classes.


In [9]:
# InceptionV3 Model

def InceptionV3Model():
  base_model = InceptionV3(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
  for layer in base_model.layers:
    layer.trainable = False
  x = base_model.output
  x = layers.GlobalAveragePooling2D()(x)
  x = layers.Dense(128, activation='relu')(x)
  x = layers.Dense(4, activation='softmax')(x)  # Four classes: Stage 1, Stage 2, Stage 3, No ROP
  model = models.Model(inputs=base_model.input, outputs=x)
  model.compile(optimizer=Adam(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  model.summary()
  r = model.fit_generator(
  train_generator,
  validation_data=test_generator,
  epochs=10,
  steps_per_epoch=len(train_generator),
  validation_steps=len(test_generator))
  test_loss, test_accuracy = model.evaluate(test_generator)
  train_loss, train_accuracy = model.evaluate(train_generator)
  print(f"InceptionV3 Train Accuracy: {train_accuracy * 100:.2f}%")
  print(f"InceptionV3 Test Accuracy: {test_accuracy * 100:.2f}%")


In [10]:
# ResNet50 Model

def ResNet50Model():
  base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
  for layer in base_model.layers:
    layer.trainable = False
  x = base_model.output
  x = layers.GlobalAveragePooling2D()(x)
  x = layers.Dense(128, activation='relu')(x)
  x = layers.Dense(4, activation='softmax')(x)  # Four classes: Stage 1, Stage 2, Stage 3, No ROP
  model = models.Model(inputs=base_model.input, outputs=x)
  model.compile(optimizer=Adam(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  model.summary()
  r = model.fit_generator(
  train_generator,
  validation_data=test_generator,
  epochs=10,
  steps_per_epoch=len(train_generator),
  validation_steps=len(test_generator))
  test_loss, test_accuracy = model.evaluate(test_generator)
  train_loss, train_accuracy = model.evaluate(train_generator)
  print(f"ResNet50 Train Accuracy: {train_accuracy * 100:.2f}%")
  print(f"ResNet50 Test Accuracy: {test_accuracy * 100:.2f}%")

In [11]:
# VGG16 Model

def VGG16Model():
  base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
  for layer in base_model.layers:
    layer.trainable = False
  x = base_model.output
  x = layers.Flatten()(x)
  x = layers.Dense(128, activation='relu')(x)
  x = layers.Dense(4, activation='softmax')(x)  # Four classes: Stage 1, Stage 2, Stage 3, No ROP
  model = models.Model(inputs=base_model.input, outputs=x)
  model.compile(optimizer=Adam(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  model.summary()
  r = model.fit_generator(
  train_generator,
  validation_data=test_generator,
  epochs=10,
  steps_per_epoch=len(train_generator),
  validation_steps=len(test_generator))
  test_loss, test_accuracy = model.evaluate(test_generator)
  train_loss, train_accuracy = model.evaluate(train_generator)
  print(f"VGG16 Train Accuracy: {train_accuracy * 100:.2f}%")
  print(f"VGG16 Test Accuracy: {test_accuracy * 100:.2f}%")



In [12]:
# VGG19 Model

def VGG19Model():
  base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
  for layer in base_model.layers:
    layer.trainable = False
  x = base_model.output
  x = layers.Flatten()(x)
  x = layers.Dense(128, activation='relu')(x)
  x = layers.Dense(4, activation='softmax')(x)  # Four classes: Stage 1, Stage 2, Stage 3, No ROP
  model = models.Model(inputs=base_model.input, outputs=x)
  model.compile(optimizer=Adam(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  model.summary()
  r = model.fit_generator(
  train_generator,
  validation_data=test_generator,
  epochs=10,
  steps_per_epoch=len(train_generator),
  validation_steps=len(test_generator))
  test_loss, test_accuracy = model.evaluate(test_generator)
  train_loss, train_accuracy = model.evaluate(train_generator)
  print(f"VGG19 Train Accuracy: {train_accuracy * 100:.2f}%")
  print(f"VGG19 Test Accuracy: {test_accuracy * 100:.2f}%")

In [13]:
# DenseNet201 Model

def DenseNet201Model():
  base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
  for layer in base_model.layers:
    layer.trainable = False
  x = base_model.output
  x = layers.GlobalAveragePooling2D()(x)
  x = layers.Dense(128, activation='relu')(x)
  x = layers.Dense(4, activation='softmax')(x)  # Four classes: Stage 1, Stage 2, Stage 3, No ROP
  model = models.Model(inputs=base_model.input, outputs=x)
  model.compile(optimizer=Adam(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  model.summary()
  r = model.fit_generator(
  train_generator,
  validation_data=test_generator,
  epochs=10,
  steps_per_epoch=len(train_generator),
  validation_steps=len(test_generator))
  test_loss, test_accuracy = model.evaluate(test_generator)
  train_loss, train_accuracy = model.evaluate(train_generator)
  print(f"DenseNet201 Train Accuracy: {train_accuracy * 100:.2f}%")
  print(f"DenseNet201 Test Accuracy: {test_accuracy * 100:.2f}%")

In [14]:
# MobileNet Model

def MobileNetModel():
  base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
  for layer in base_model.layers:
    layer.trainable = False
  x = base_model.output
  x = layers.GlobalAveragePooling2D()(x)
  x = layers.Dense(128, activation='relu')(x)
  x = layers.Dense(4, activation='softmax')(x)  # Four classes: Stage 1, Stage 2, Stage 3, No ROP
  model = models.Model(inputs=base_model.input, outputs=x)
  model.compile(optimizer=Adam(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  model.summary()
  r = model.fit_generator(
  train_generator,
  validation_data=test_generator,
  epochs=10,
  steps_per_epoch=len(train_generator),
  validation_steps=len(test_generator))
  test_loss, test_accuracy = model.evaluate(test_generator)
  train_loss, train_accuracy = model.evaluate(train_generator)
  print(f"MobileNet Train Accuracy: {train_accuracy * 100:.2f}%")
  print(f"MobileNet Test Accuracy: {test_accuracy * 100:.2f}%")


In [15]:
# AlexNet Model
def AlexNetModel():
  model = models.Sequential()

  model.add(layers.Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=(224, 224, 3)))
  model.add(layers.MaxPooling2D((3, 3), strides=(2, 2)))
  model.add(layers.Conv2D(256, (5, 5), activation='relu'))
  model.add(layers.MaxPooling2D((3, 3), strides=(2, 2)))
  model.add(layers.Conv2D(384, (3, 3), activation='relu'))
  model.add(layers.Conv2D(384, (3, 3), activation='relu'))
  model.add(layers.Conv2D(256, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((3, 3), strides=(2, 2)))

  model.add(layers.Flatten())
  model.add(layers.Dense(4096, activation='relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(4096, activation='relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(4, activation='softmax'))  # Four classes: Stage 1, Stage 2, Stage 3, No ROP

  model.compile(optimizer=Adam(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  model.summary()

  r = model.fit_generator(
  train_generator,
  validation_data=test_generator,
  epochs=10,
  steps_per_epoch=len(train_generator),
  validation_steps=len(test_generator))
  test_loss, test_accuracy = model.evaluate(test_generator)
  train_loss, train_accuracy = model.evaluate(train_generator)
  print(f"AlexNet Train Accuracy: {train_accuracy * 100:.2f}%")
  print(f"AlexNet Test Accuracy: {test_accuracy * 100:.2f}%")


In [16]:
# Define the CNN model within the strategy scope
with strategy.scope():
    input_shape = (224, 224, 3)
    inputs = Input(shape=input_shape)
    x = Conv2D(64, (3, 3), activation='relu')(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(256, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(4, activation='softmax')(x)  # 4 classes (No_ROP, Stage_1, Stage_2, Stage_3)

    model = Model(inputs=inputs, outputs=output)

    # Compile the model
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model using the available GPU (T4 GPU)
with strategy.scope():
    history = model.fit(
        train_generator,
        validation_data=test_generator,
        epochs=10,
        steps_per_epoch=len(train_generator),
        validation_steps=len(test_generator)
    )

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Epoch 1/10
22/22 [==============================] - 30s 624ms/step - loss: 1.6780 - accuracy: 0.2957 - val_loss: 1.2124 - val_accuracy: 0.4767
Epoch 2/10
22/22 [==============================] - 12s 560ms/step - loss: 1.0819 - accuracy: 0.5257 - val_loss: 0.9556 - val_accuracy: 0.6200
Epoch 3/10
22/22 [==============================] - 12s 560ms/step - loss: 0.8937 - accuracy: 0.5986 - val_loss: 0.7931 - val_accuracy: 0.6467
Epoch 4/10
22/22 [==============================] - 12s 553ms/step - loss: 0.7054 - accuracy: 0.7257 - val_loss: 0.7222 - val_accuracy: 0.7200
Epoch 5/10
22/22 [==============================] - 12s 567ms/step - loss: 0.5486 - accuracy: 0.8043 - val_loss: 0.7117 - val_accuracy: 0.7200
Epoch 6/10
22/22 [==============================] - 12s 556ms/step - loss: 0.5000 - accuracy: 0.8114 - val_loss: 0.5947 - val_accuracy: 0.7667
Epoch 7/10
22/22 [==============================] - 12s 562ms/step - loss: 0.4219 - accuracy: 0.8371 - val_loss: 0.5179 - val_accuracy: 0.8200